In [1]:
import sys
import numpy as np
import pandas as pd
import simtk.openmm
import os
import glob
import shutil
import time
import MDAnalysis as mda
import math
pd.set_option("display.precision", 10)

ca_sbm_3spn_openmm_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM'
sys.path.insert(0, ca_sbm_3spn_openmm_path)

import openFiber.open3SPN2.ff3SPN2 as ff3SPN2
import openFiber.calphaSBM.ffCalpha as ffCalpha
import openFiber.openFiberTools as openFiberTools
import openFiber.rigid

# set some global parameters
n_nucl = 2
scale_factor = 2.5 # scale factor for all the SBM related potentials
run_smog = False
apply_rigid_body = False
compare_with_lammps = True
smog_dir = '/Users/administrator/Documents/Tools/smog-2.2' # the directory where smog is installed
# smog_dir does not matter if run_smog == False
histone_dna_data_dir = '%s/separate-histone-dna-pdb/%dmer-separate-output' % (ca_sbm_3spn_openmm_path, n_nucl) # the path that saves the input pdb structures for all the dna and histones (each histone is saved in one pdb file)
group_rigid_txt_path = '%s/data/chromatin-%dmer-rigid-group/group_rigid.txt' % (ca_sbm_3spn_openmm_path, n_nucl) # group_rigid.txt file with atom index starts from 1 (lammps format)
main_output_dir = '%s/files-smliu/chromatin-%dmer' % (ca_sbm_3spn_openmm_path, n_nucl) # the main output directory
smog_output_dir = '%s/smog' % main_output_dir # smog output directory
openmm_files_dir = '%s/openmm-files' % main_output_dir
platform_name = 'CPU' # 'Reference', 'CPU', 'CUDA', 'OpenCL'
sim_output_dir = '%s/sim-test-%s' % (openmm_files_dir, platform_name)

# build the output directories
if not os.path.exists(main_output_dir):
    os.makedirs(main_output_dir)
if not os.path.exists(smog_output_dir):
    os.makedirs(smog_output_dir)
if not os.path.exists(openmm_files_dir):
    os.makedirs(openmm_files_dir)
if not os.path.exists(sim_output_dir):
    os.makedirs(sim_output_dir)

# 1 Build the CG model for the chromatin

## 1.1 Load PDB structures

In [2]:
# load each histone
all_histone_fix_list = []
for i in range(n_nucl):
    all_histone_fix_list.append(ff3SPN2.fixPDB('%s/histone-%d.pdb' % (histone_dna_data_dir, i + 1)))

# load dna
dna_fix = ff3SPN2.fixPDB('%s/dna.pdb' % histone_dna_data_dir)

# convert to pandas format tables that includes all the information of each histone and dna
# the reason we use pandas table is because there is no length limit for the entries
all_histone_atom_tables = []
for each in all_histone_fix_list:
    all_histone_atom_tables.append(ff3SPN2.pdb2table(each))

dna_atom_table = ff3SPN2.pdb2table(dna_fix)

# update serial for each histone and dna
for i in range(len(all_histone_atom_tables)):
    all_histone_atom_tables[i] = openFiberTools.change_serial_resSeq(all_histone_atom_tables[i], change_resSeq=False)
dna_atom_table = openFiberTools.change_serial_resSeq(dna_atom_table, change_resSeq=False)

# combine the tables for histones and DNA
complex_table = all_histone_atom_tables[0]
for i in range(1, len(all_histone_atom_tables)):
    complex_table = openFiberTools.combine_molecules(complex_table, all_histone_atom_tables[i], add_resSeq=False)
complex_table = openFiberTools.combine_molecules(complex_table, dna_atom_table, add_resSeq=False)

# write the data into csv file
output_dir = '%s/all-atom-fiber' % main_output_dir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
complex_table.to_csv('%s/chromatin-%dmer.csv' % (output_dir, n_nucl), index=False)


## 1.2 Apply SMOG to histones

In [3]:
# write all the histones into a PDB file
ffCalpha.writePDB_protein(complex_table, '%s/histones.pdb' % smog_output_dir)

# add TER to the pdb file
input_pdb_path = '%s/histones.pdb' % smog_output_dir
output_pdb_path = '%s/histones_clean.pdb' % smog_output_dir
openFiberTools.add_ter_end_and_remove_OXT_for_pdb(input_pdb_path, output_pdb_path)

In [4]:
if run_smog:
    # perform smog on the clean protein pdb file
    cmd = 'source %s/configure.smog2; ' % smog_dir
    cmd = cmd + 'cd %s; ' % smog_output_dir
    sbm_aa_path = '%s/share/templates/SBM_AA' % smog_dir
    sbm_calpha_gaussian_path = '%s/share/templates/SBM_calpha+gaussian' % smog_dir
    cmd = cmd + 'smog2 -i histones_clean.pdb -t %s -tCG %s' % (sbm_aa_path, sbm_calpha_gaussian_path)
    print(cmd)
    os.system(cmd)

    # pick out sections from smog.top
    cmd = 'cd %s; ' % smog_output_dir
    py_get_section_script_path = '%s/openFiber/getSection.py' % ca_sbm_3spn_openmm_path
    key_word_list = ['atoms', 'bonds', 'angles', 'dihedrals', 'pairs', 'exclusions', 'system']
    for i in range(len(key_word_list) - 1):
        keyword1 = key_word_list[i]
        keyword2 = key_word_list[i + 1]
        cmd = cmd + 'python %s ./smog.top %s.dat "[ %s ]" "[ %s ]"; ' % (py_get_section_script_path, keyword1, keyword1, keyword2)
    print(cmd)
    os.system(cmd)


## 1.3 Load DNA and histone CG models separately and then combine them

In [5]:
# generate DNA and protein CG model from complex_table
cg_dna = ff3SPN2.DNA.CoarseGrain(complex_table)
cg_proteins = ffCalpha.Protein.CoarseGrain(complex_table)

# combine CG histones and DNA
cg_fiber = pd.concat([cg_proteins, cg_dna], sort=False)
cg_fiber.index = list(range(len(cg_fiber.index)))
cg_fiber['serial'] = list(range(len(cg_fiber.index)))

# change the chainID of the chromatin fiber
cg_fiber_unique_chainID = openFiberTools.change_unique_chainID(cg_fiber)

# save protein sequence
if not os.path.exists('%s/cg-fiber' % main_output_dir):
    os.makedirs('%s/cg-fiber' % main_output_dir)
protein_seq_path = '%s/cg-fiber/protein_seq.txt' % main_output_dir
ffCalpha.save_protein_sequence(cg_fiber_unique_chainID, sequence_file=protein_seq_path)

# write cg_fiber to pdb format, which will later be loaded by openmm
# note we convert cg_fiber instead of cg_fiber_unique_chainID to pdb format, since cg_fiber_unique_chainID may have chainID length beyond the limit of pdb format
cg_fiber_pdb_path = '%s/cg-fiber/cg_fiber.pdb' % main_output_dir
#print(cg_fiber)
ffCalpha.writePDB(cg_fiber, cg_fiber_pdb_path)
cg_fiber.to_csv('%s/cg-fiber/cg_fiber.csv' % main_output_dir, index=False)

# also save cg_fiber_unique_chainID
cg_fiber_unique_chainID.to_csv('%s/cg-fiber/cg_fiber_unique_chainID.csv' % main_output_dir, index=False)

# 2 Set up OpenMM simulations

## 2.1 Set up the system, protein and dna objects

In [6]:
ffCalpha_xml_path = '%s/openFiber/calphaSBM/ffCalpha.xml' % ca_sbm_3spn_openmm_path
cg_fiber_pdb_path = '%s/cg-fiber/cg_fiber.pdb' % main_output_dir
os.chdir('%s/cg-fiber' % main_output_dir)

pdb = simtk.openmm.app.PDBFile(cg_fiber_pdb_path)
top = pdb.getTopology()
coord = pdb.getPositions(asNumpy=True)
forcefield = simtk.openmm.app.ForceField(ffCalpha_xml_path, ff3SPN2.xml)
s = forcefield.createSystem(top)

In [7]:
# create the DNA and protein objects
dna = ff3SPN2.DNA.fromCoarsePDB_through_pdframe(pd_frames=cg_fiber_unique_chainID, dna_type='B_curved', compute_topology=True)
with open(protein_seq_path, 'r') as ps:
    protein_seq = ps.readlines()[0].rstrip()
protein = ffCalpha.Protein.fromCoarsePDB_through_pdframe(pd_frames=cg_fiber_unique_chainID, sequence=protein_seq)

dna.periodic = False
protein.periodic = False


Empty DataFrame
Columns: [recname_old, serial_old, name_old, altLoc_old, resname_old, chainID_old, resSeq_old, iCode_old, x_old, y_old, z_old, occupancy_old, tempFactor_old, element_old, charge_old, real_resname, type_old, recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []

[0 rows x 33 columns]
[134, 1210, 1338, 1460, 1595, 1697, 1825, 1947, 236, 364, 486, 621, 723, 851, 973, 1108]


## 2.2 Set up forces

### 2.2.1 Set up rigid body list and chain list

In [8]:
# create rigid identity list for the fiber
rigid_body_array = np.loadtxt(group_rigid_txt_path, dtype=int) - 1 # atom index starts from 0
fiber_rigid_identity = []
n_cg_atoms = cg_fiber.shape[0]
print('n_cg_atoms = %d' % n_cg_atoms)
for i in range(n_cg_atoms):
    rigid_identity = None
    if apply_rigid_body:
        for j in range(n_nucl):
            if i in rigid_body_array[j]:
                rigid_identity = j
                break
    fiber_rigid_identity.append(rigid_identity)

histones_chains = openFiberTools.get_single_fiber_histones_chains(n_nucl)

n_cg_atoms = 3830


### 2.2.2 Set up forces for histones and dna

In [9]:
# load the force parameters given by smog
smog_atoms_file_path = '%s/atoms.dat' % smog_output_dir
smog_bonds_file_path = '%s/bonds.dat' % smog_output_dir
smog_angles_file_path = '%s/angles.dat' % smog_output_dir
smog_dihedrals_file_path = '%s/dihedrals.dat' % smog_output_dir
smog_exclusions_file_path = '%s/exclusions.dat' % smog_output_dir
smog_pairs_file_path = '%s/pairs.dat' % smog_output_dir

smog_bonds_data = openFiberTools.load_smog_bonds(smog_bonds_file_path)
smog_angles_data = openFiberTools.load_smog_angles(smog_angles_file_path)
smog_dihedrals_data = openFiberTools.load_smog_dihedrals(smog_dihedrals_file_path)
smog_pairs_data = openFiberTools.load_smog_pairs(smog_pairs_file_path)

smog_data = dict(bonds=smog_bonds_data, angles=smog_angles_data, dihedrals=smog_dihedrals_data, pairs=smog_pairs_data)

# set force dictionary
forces = {}

# set exclusions list
protein_exclusions_list, dna_exclusions_list = openFiberTools.buildNonBondedExclusionsList(protein, dna, smog_exclusions_file_path, fiber_rigid_identity)

if compare_with_lammps:
    print('Compare with lammps, so we remove protein native pairs from protein_exclusions_list')
    print('Before removing protein native pairs, total number of exclusions between protein atoms is %d' % len(protein_exclusions_list))
    # for comparison with lammps, we do not put protein native pairs in protein_exclusions_list
    smog_exclusions = np.loadtxt(smog_exclusions_file_path, comments=';') - 1 # let atom index start from 0
    if smog_exclusions.ndim == 1:
        smog_exclusions = np.reshape(smog_exclusions, (1, -1))
    smog_exclusions_list = []
    for i in range(smog_exclusions.shape[0]):
        i1, i2 = int(smog_exclusions[i, 0]), int(smog_exclusions[i, 1])
        if i1 > i2:
            i1, i2 = i2, i1
        smog_exclusions_list.append((i1, i2))
    protein_exclusions_list_compare_with_lammps = []
    for each in protein_exclusions_list:
        if each not in smog_exclusions_list:
            protein_exclusions_list_compare_with_lammps.append(each)
    protein_exclusions_list = protein_exclusions_list_compare_with_lammps
    print('After removing protein native pairs, total number of exclusions between protein atoms is %d' % len(protein_exclusions_list))
else:
    print('total number of exclusions between protein atoms is %d' % len(protein_exclusions_list))

print('total number of protein native pairs is %d' % len(smog_exclusions_list))
print('total number of exclusions between DNA atoms is %d' % len(dna_exclusions_list))


# add DNA-DNA, protein-DNA and protein-protein interactions
openFiberTools.add_dna_protein_forces(s, forces, dna, protein, smog_data, dna_exclusions_list, protein_exclusions_list, fiber_rigid_identity, scale_factor) # for simulations



Compare with lammps, so we remove protein native pairs from protein_exclusions_list
Before removing protein native pairs, total number of exclusions between protein atoms is 11119
After removing protein native pairs, total number of exclusions between protein atoms is 5748
total number of protein native pairs is 5371
total number of exclusions between DNA atoms is 57486
Add DNA-DNA forces
force_name = Bond
force name Bond is updated to BondDNA
force_name = Angle
force name Angle is updated to AngleDNA
force_name = Stacking
force_name = Dihedral
force name Dihedral is updated to DihedralDNA
force_name = BasePair
force_name = CrossStacking
force_name = Exclusion
Number of exclusions is 63234
force name Exclusion is updated to ExclusionDNA_DNA
force_name = Electrostatics
Number of exclusions is 63234
force name Electrostatics is updated to ElectrostaticsDNA_DNA
Add protein-DNA forces
force_name = ExclusionProteinDNA
force_name = ElectrostaticsProteinDNA
Add protein-protein forces


## 2.3 Set up rigid body

In [10]:
rigid_body_array = np.loadtxt(group_rigid_txt_path, dtype=int) - 1 # atom index starts from 0
rigid_body_list = []
for i in range(n_nucl):
    new_list = rigid_body_array[i].tolist()
    new_list = [int(each) for each in new_list]
    rigid_body_list.append(new_list)
openFiber.rigid.createRigidBodies(s, coord, rigid_body_list)

## 2.4 Run the simulation

In [11]:
temperature = 300*simtk.openmm.unit.kelvin

integrator = simtk.openmm.LangevinIntegrator(temperature, 1/simtk.openmm.unit.picosecond, 10*simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)

simulation = simtk.openmm.app.Simulation(top, s, integrator, platform)

simulation.context.setPositions(coord)
#energy_unit=simtk.openmm.unit.kilojoule_per_mole
energy_unit = simtk.openmm.unit.kilocalories_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print("The unRESCALED energy is %.6f %s" % (energy, energy_unit.get_symbol()))

# get the detailed energy after the simulation
# double check SBM pair, nonbonded, and electrostatic interactions
energies = {}
for force_name, force in forces.items():
    group = force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups={group})
    energies[force_name] = state.getPotentialEnergy().value_in_unit(energy_unit)
    print('force group %d, force name %s, energy %.6f %s' % (group, force_name, energies[force_name], energy_unit.get_symbol()))

The unRESCALED energy is 30316.217933 kcal/mol
force group 6, force name BondDNA, energy 794.973379 kcal/mol
force group 7, force name AngleDNA, energy 4022.794150 kcal/mol
force group 8, force name Stacking, energy -85.480694 kcal/mol
force group 9, force name DihedralDNA, energy -1491.847922 kcal/mol
force group 10, force name BasePair, energy -797.361919 kcal/mol
force group 11, force name CrossStacking, energy -184.953772 kcal/mol
force group 12, force name ExclusionDNA_DNA, energy 68.916876 kcal/mol
force group 13, force name ElectrostaticsDNA_DNA, energy 107.558095 kcal/mol
force group 14, force name ExclusionProteinDNA, energy 31590.688928 kcal/mol
force group 15, force name ElectrostaticsProteinDNA, energy -378.163878 kcal/mol
force group 19, force name SBM_Bond, energy 0.000000 kcal/mol
force group 20, force name SBM_Angle, energy 0.000000 kcal/mol
force group 21, force name SBM_Dihedral, energy 0.000000 kcal/mol
force group 22, force name SBM_Pair, energy -3209.249522 kcal/mo

In [12]:
# Now we will save a serialization of this simulation into OpenMM's native XML format
# We can re-initialize the system later for further simulations without all of the bothersome set-up by loading these files!
# We'll write exactly the same XML files Folding@home uses to transfer simulation data for restarts to/from users
state = simulation.context.getState(getPositions=True, getVelocities=True, getForces=True, getEnergy=True, getParameters=True, enforcePeriodicBox=False)

# system.xml contains all of the force field parameters
with open('%s/system.xml' % openmm_files_dir, 'w') as f:
    system_xml = simtk.openmm.XmlSerializer.serialize(s) 
    f.write(system_xml)
    # integrator.xml contains the configuration for the integrator, RNG seed
with open('%s/integrator.xml' % openmm_files_dir, 'w') as f:
    integrator_xml = simtk.openmm.XmlSerializer.serialize(integrator) 
    f.write(integrator_xml)
    # state.xml contains positions, velocities, forces, the barostat
with open('%s/state.xml' % openmm_files_dir, 'w') as f: 
    f.write(simtk.openmm.XmlSerializer.serialize(state))

# there is also a binary "Checkpoint" file
# using a "Checkpoint" file only work on the same hardware+software combination. 
simulation.saveState('%s/state_serialized.xml' % openmm_files_dir)
simulation.saveCheckpoint('%s/checkpnt.chk' % openmm_files_dir)


In [13]:
# do the energy minimization
start_time = time.time()
simulation.minimizeEnergy()
end_time = time.time()
delta_time = end_time - start_time
print('energy minimization time cost: %.2f' % delta_time)
energy_unit = simtk.openmm.unit.kilocalories_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('energy minimized')
print('energy = %.6f %s' % (energy, energy_unit.get_symbol()))
for force_name, force in forces.items():
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups={group})
    energies[force_name] = state.getPotentialEnergy().value_in_unit(energy_unit)
    print('force group %d, force name %s, energy %.6f %s' % (group, force_name, energies[force_name], energy_unit.get_symbol()))

energy minimization time cost: 125.12
energy minimized
energy = -4240.803232 kcal/mol
force group 6, force name BondDNA, energy 309.558920 kcal/mol
force group 7, force name AngleDNA, energy 2128.137879 kcal/mol
force group 8, force name Stacking, energy -483.506631 kcal/mol
force group 9, force name DihedralDNA, energy -1692.470229 kcal/mol
force group 10, force name BasePair, energy -945.608672 kcal/mol
force group 11, force name CrossStacking, energy -256.205072 kcal/mol
force group 12, force name ExclusionDNA_DNA, energy 32.857966 kcal/mol
force group 13, force name ElectrostaticsDNA_DNA, energy 107.690893 kcal/mol
force group 14, force name ExclusionProteinDNA, energy 251.022538 kcal/mol
force group 15, force name ElectrostaticsProteinDNA, energy -366.108759 kcal/mol
force group 19, force name SBM_Bond, energy 0.145014 kcal/mol
force group 20, force name SBM_Angle, energy 2.508176 kcal/mol
force group 21, force name SBM_Dihedral, energy 4.486188 kcal/mol
force group 22, force name

In [14]:
# run simulation
simulation.context.setVelocitiesToTemperature(temperature)

# add simulation reporters
dcd_reporter = simtk.openmm.app.DCDReporter('%s/output.dcd' % sim_output_dir, 100)
energy_reporter = simtk.openmm.app.StateDataReporter(sys.stdout, 100, step=True, time=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, temperature=True, speed=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(energy_reporter)
start_time = time.time()
n_steps = 200
simulation.step(n_steps)
end_time = time.time()
delta_time = end_time - start_time
print('simulation takes %.2f seconds for %d steps' % (delta_time, n_steps))


#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,-13306.25621693386,4726.702077330317,-8579.554139603544,244.67036385877455,0
200,2.0000000000000013,-12805.311548195703,5412.277850536156,-7393.033697659547,280.1581249105086,2.78
simulation takes 61.92 seconds for 200 steps
